In [399]:
from IPython.display import Image
from IPython.core.display import HTML 
#from __future__ import print_func  tion, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import json
from sklearn.model_selection import train_test_split
Image(url= "https://cdn-images-1.medium.com/max/1600/1*UkI9za9zTR-HL8uM15Wmzw.png")

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 500)


In [440]:
# the columns use for predictions
# NOTE:
# When adding more features
# make sure make the model in a differnt path because it screws with the checkpoints
saved_model = "./6_col_checkpoints/"
features = ['posX', 'posY', 'posZ', 'velX', 'velY', 'velZ']
col_numbers = len(features)


In [441]:
# ------------- Hyperparameters ---------------------


# hyperparams

num_epoch = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4 # number of layers

number_classes = 2
echo_step = 3
batch_size = 5
number_classes = total_series_length // batch_size // truncated_backprop_length
timestep = 1000

In [442]:
# copying the dataframe so I have a before and after
df_cir_hor_cclk = pd.read_csv('c:/Users/Franz Lom/Documents/TensorFlow/gesture_recognition/Data/extracted_data/cir_hor_cclk.csv') # saving circle horizontal counter clockwise csv as df
df_v_shape = pd.read_csv('c:/Users/Franz Lom/Documents/TensorFlow/gesture_recognition/Data/extracted_data/v_shape.csv')
#df_cir_hor_cclk['shape'] = 1
#df_v_shape['shape'] = 2

df_cir = df_cir_hor_cclk.copy()
df_v = df_v_shape.copy()

df_cir['shape'] = 1
df_v['shape'] = 2
list(df_cir_hor_cclk)


['tester',
 'trial',
 'rightHanded',
 'timestamp',
 'posX',
 'posY',
 'posZ',
 'posRelX',
 'posRelY',
 'posRelZ',
 'velX',
 'velY',
 'velZ',
 'quatW',
 'quatX',
 'quatY',
 'quatZ',
 'accRelX',
 'accRelY',
 'accRelZ',
 'accAbsX',
 'accAbsY',
 'accAbsZ']

In [443]:
def pad_features(not_pad_features):
    """
    A function that returns a padded array with a size 300
    :param not_pad_features: 
    :return: an np.array of padded vector with the length of  300 
    """
    
    list_pad_vec = list()
    feature_len = [len(s) for s in not_pad_features]
    if max(feature_len) > 300:
        raise Exception('the longest stroke is greater than 300. len: [}'
                        .format(max(feature_len)))
    padding = np.zeros((300, col_numbers))
    
    for x in not_pad_features:
        padding[:x.shape[0], :x.shape[1]] = x 
        list_pad_vec.append(padding)
        padding = np.zeros((300, col_numbers))
        
    return(np.array(list_pad_vec))

In [444]:
def group_drawn_shape(grouped):
    """
    
    :param grouped: grouped value from the dataframe, ['trial'. 'tester']
    :return: np.array of grouped vectors
    """
    source = [x for x in grouped]
    list_vec = list()
    for v in source:
        df = v[1]
        df = df[features]
        x = df.values
        list_vec.append(x)
    return np.array(list_vec)

In [445]:
cir_features = group_drawn_shape(df_cir.groupby(['trial', 'tester']))
v_features = group_drawn_shape(df_v.groupby(['trial', 'tester']))

cir_stroke_length = [len(stroke) for stroke in cir_features]
cir_stroke = [stroke for stroke in cir_features]
print(cir_features.shape)
pad_cir_features = pad_features(cir_features)
pad_v_features = pad_features(v_features)

(280,)


In [446]:
print(pad_cir_features.shape)
print(pad_v_features.shape)

#combine the padded data so we have one data to work with
data = np.vstack((pad_cir_features, pad_v_features))
data.shape


(280, 300, 6)
(280, 300, 6)


(560, 300, 6)

In [447]:
# labels are one hot encoded
horizontal_circle_label = np.zeros((280,2))
v_shape_label = np.zeros((280,2))

horizontal_circle_label[:, 0] = 1
v_shape_label[:, 1] = 1

labels = np.vstack((horizontal_circle_label, v_shape_label))
labels[275:285]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [448]:
horizontal_circle_label = np.zeros(280)
v_shape_label = np.zeros(280)
v_shape_label[:] = 1
labels = np.concatenate([horizontal_circle_label, v_shape_label])
labels[275:285]

array([0., 0., 0., 0., 0., 1., 1., 1., 1., 1.])

In [449]:
# split the data to training training and testing data.
# x = features
# y = labels
x_train, x_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.20, random_state=42)

print(x_train.shape)
print(y_train.shape)

(448, 300, 6)
(448,)


In [450]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": x_train},
    y = y_train,
    num_epochs=1,
    shuffle=False
)

In [451]:
train_input_fn

<function tensorflow.python.estimator.inputs.numpy_io.numpy_input_fn.<locals>.input_fn>

In [452]:
train_input_fn()

({'x': <tf.Tensor 'fifo_queue_DequeueUpTo_10:1' shape=(?, 300, 6) dtype=float64>},
 <tf.Tensor 'fifo_queue_DequeueUpTo_10:2' shape=(?,) dtype=float64>)

In [453]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=np.array(y_test),
    num_epochs=1,
    shuffle=False
)

In [454]:
some_gesture =x_test[0:9]
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": some_gesture},
    num_epochs=1,
    shuffle=False
)

In [455]:
some_gesture_labels = y_test[0:9]

In [456]:
features_x = tf.feature_column.numeric_column(key="x", shape=(300, col_numbers))
feature_columns = [features_x]

In [457]:
# IGNORE FOR NOW
# this model is using essentially 1 giant feature (posX, posY, posZ)

#keys = ['posX', 'posY', 'posZ']
#feature_columns = list()
#for k in keys:
#    feature_columns.append(tf.feature_column.numeric_column(key=k))


#feature_columns[0]

In [458]:
num_hidden_units = [512, 256, 128]

In [459]:
model = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                   hidden_units=num_hidden_units,
                                   activation_fn=tf.nn.relu,
                                   n_classes=2,
                                   model_dir=savd_model)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000026E34799780>, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_is_chief': True, '_keep_checkpoint_max': 5, '_train_distribute': None, '_evaluation_master': '', '_service': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_device_fn': None, '_task_id': 0, '_global_id_in_cluster': 0, '_save_summary_steps': 100, '_master': '', '_num_worker_replicas': 1, '_tf_random_seed': None, '_session_config': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './6_col_checkpoints/'}


In [460]:
model.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Restoring parameters from ./6_col_checkpoints/model.ckpt-4


...

INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 4 into ./6_col_checkpoints/model.ckpt.


INFO:tensorflow:loss = 61.262955, step = 5


INFO:tensorflow:Saving checkpoints for 8 into ./6_col_checkpoints/model.ckpt.


INFO:tensorflow:Loss for final step: 26.504456.


In [461]:
result = model.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.

INFO:tensorflow:Starting evaluation at 2018-11-12-02:16:12


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Restoring parameters from ./6_col_checkpoints/model.ckpt-8


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Finished evaluation at 2018-11-12-02:16:13


INFO:tensorflow:Saving dict for global step 8: accuracy = 0.76785713, accuracy_baseline = 0.55357146, auc = 0.97709674, auc_precision_recall = 0.9760859, average_loss = 0.4094889, global_step = 8, label/mean = 0.44642857, loss = 45.862755, precision = 0.6621622, prediction/mean = 0.60119015, recall = 0.98


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8: ./6_col_checkpoints/model.ckpt-8


In [462]:
result

{'accuracy': 0.76785713,
 'accuracy_baseline': 0.55357146,
 'auc': 0.97709674,
 'auc_precision_recall': 0.9760859,
 'average_loss': 0.4094889,
 'global_step': 8,
 'label/mean': 0.44642857,
 'loss': 45.862755,
 'precision': 0.6621622,
 'prediction/mean': 0.60119015,
 'recall': 0.98}

In [463]:
print("Classification Accuracy for data set (posX, posY, posZ): "
      "{0:.2%}".format(result["accuracy"]))

Classification Accuracy for data set (posX, posY, posZ): 76.79%


In [464]:
# predictions!

predictions = model.predict(input_fn=predict_input_fn)


In [465]:
cls = [p['classes'] for p in predictions]
cls

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Restoring parameters from ./6_col_checkpoints/model.ckpt-8


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


[array([b'1'], dtype=object),
 array([b'1'], dtype=object),
 array([b'1'], dtype=object),
 array([b'1'], dtype=object),
 array([b'1'], dtype=object),
 array([b'0'], dtype=object),
 array([b'0'], dtype=object),
 array([b'1'], dtype=object),
 array([b'0'], dtype=object)]

In [466]:
cls_pred = np.array(cls, dtype='int').squeeze()
cls_pred

array([1, 1, 1, 1, 1, 0, 0, 1, 0])

In [438]:
some_gesture  # gestures to be predicted
print("the prediction is: \t",cls_pred)      # predicted gestures
print("the actual is: \t\t ",y_test[0:9])

the prediction is: 	 [1 1 0 1 1 0 0 1 0]
the actual is: 		  [1. 1. 0. 0. 1. 0. 0. 1. 0.]


In [439]:
# FINISHED!